# Building Agentic Applications with the Responses API

This notebook presents practical, reusable patterns and functions for building production-ready agentic applications using OpenAI's Responses API. Each pattern addresses common challenges in developing chat agents and agentic workflows, with complete working examples you can adapt for your own projects.

## What You'll Learn

- **Streaming responses** with real-time callbacks
- **Retry logic** with exponential backoff for reliability
- **Token budget management** to control costs
- **Multi-turn conversations** with context management
- **Error handling** patterns for production systems
- **Function calling** for tool integration
- **Structured outputs** with validation
- **Cost tracking** for budget monitoring
- **Context window management** for long conversations
- **Response caching** for efficiency

## Setup

In [ ]:
from openai import OpenAI
import os
import json
import time
from typing import Optional, Callable, Dict, List, Any
from dataclasses import dataclass
from datetime import datetime

# Initialize the client
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

## 1. Streaming Response Handler

When building interactive agents, users expect real-time feedback. This pattern shows how to stream responses with callbacks for different events (content, completion, errors). This is essential for chat interfaces where you want to display text as it's generated.

In [ ]:
def stream_response(
    client: OpenAI,
    model: str,
    input_text: str,
    on_content: Optional[Callable[[str], None]] = None,
    on_complete: Optional[Callable[[Dict], None]] = None,
    on_error: Optional[Callable[[Exception], None]] = None,
    **kwargs
) -> str:
    """
    Stream a response with callbacks for real-time updates.
    
    Args:
        client: OpenAI client instance
        model: Model to use
        input_text: Input prompt
        on_content: Callback for each content chunk (receives text)
        on_complete: Callback when stream completes (receives final response)
        on_error: Callback for errors (receives exception)
        **kwargs: Additional parameters for responses.create()
    
    Returns:
        Complete response text
    """
    full_text = ""
    
    try:
        stream = client.responses.create(
            model=model,
            input=input_text,
            stream=True,
            **kwargs
        )
        
        for chunk in stream:
            # Extract text from the chunk
            if hasattr(chunk, 'output') and chunk.output:
                for output_item in chunk.output:
                    if hasattr(output_item, 'content'):
                        for content_item in output_item.content:
                            if hasattr(content_item, 'text'):
                                text = content_item.text
                                full_text += text
                                if on_content:
                                    on_content(text)
        
        if on_complete:
            on_complete({"text": full_text, "status": "completed"})
        
        return full_text
    
    except Exception as e:
        if on_error:
            on_error(e)
        raise

# Example usage
def print_chunk(text: str):
    print(text, end='', flush=True)

def on_done(response: Dict):
    print(f"\n\n✓ Complete! Total length: {len(response['text'])} chars")

print("Agent: ", end='')
response_text = stream_response(
    client,
    model="gpt-5-mini",
    input_text="Explain how neural networks learn in 3 sentences.",
    on_content=print_chunk,
    on_complete=on_done
)

## 2. Retry Logic with Exponential Backoff

Production systems need to handle transient failures gracefully. This pattern implements exponential backoff for retrying failed requests - essential for building reliable agents that can recover from temporary API issues, rate limits, or network problems.

In [ ]:
from openai import APIError, RateLimitError, APITimeoutError

def create_response_with_retry(
    client: OpenAI,
    max_retries: int = 3,
    initial_delay: float = 1.0,
    exponential_base: float = 2.0,
    **kwargs
):
    """
    Create a response with automatic retry logic.
    
    Args:
        client: OpenAI client
        max_retries: Maximum number of retry attempts
        initial_delay: Initial delay in seconds
        exponential_base: Multiplier for exponential backoff
        **kwargs: Parameters for responses.create()
    
    Returns:
        Response object
    """
    retryable_errors = (RateLimitError, APITimeoutError, APIError)
    
    for attempt in range(max_retries + 1):
        try:
            response = client.responses.create(**kwargs)
            return response
        
        except retryable_errors as e:
            if attempt == max_retries:
                print(f"❌ Max retries ({max_retries}) exceeded")
                raise
            
            delay = initial_delay * (exponential_base ** attempt)
            print(f"⚠️  Attempt {attempt + 1} failed: {type(e).__name__}")
            print(f"   Retrying in {delay:.1f}s...")
            time.sleep(delay)
        
        except Exception as e:
            # Non-retryable error
            print(f"❌ Non-retryable error: {type(e).__name__}")
            raise

# Example usage
response = create_response_with_retry(
    client,
    model="gpt-5-mini",
    input="What is machine learning?",
    max_retries=3
)

print(f"✓ Success! Response: {response.output_text[:100]}...")

## 3. Token Budget Manager

Managing costs is critical in production. This pattern helps you enforce token budgets, track usage across multiple requests, and prevent unexpected costs - essential for any agent that makes multiple API calls.

In [ ]:
@dataclass
class TokenBudget:
    """Track token usage against a budget."""
    max_tokens: int
    used_tokens: int = 0
    
    def can_spend(self, tokens: int) -> bool:
        """Check if we can spend the specified tokens."""
        return self.used_tokens + tokens <= self.max_tokens
    
    def spend(self, tokens: int) -> None:
        """Record token usage."""
        self.used_tokens += tokens
    
    def remaining(self) -> int:
        """Get remaining token budget."""
        return self.max_tokens - self.used_tokens
    
    def usage_percentage(self) -> float:
        """Get budget usage as percentage."""
        return (self.used_tokens / self.max_tokens) * 100

class BudgetedClient:
    """OpenAI client wrapper with token budget enforcement."""
    
    def __init__(self, client: OpenAI, budget: TokenBudget):
        self.client = client
        self.budget = budget
    
    def create_response(self, **kwargs):
        """Create response while enforcing budget."""
        # Estimate tokens for this request
        estimated = self._estimate_tokens(kwargs)
        
        if not self.budget.can_spend(estimated):
            raise ValueError(
                f"Token budget exceeded. Remaining: {self.budget.remaining()}, "
                f"Needed: {estimated}"
            )
        
        # Create response
        response = self.client.responses.create(**kwargs)
        
        # Track actual usage
        actual_tokens = response.usage.total_tokens
        self.budget.spend(actual_tokens)
        
        print(f"📊 Tokens used: {actual_tokens} | "
              f"Remaining: {self.budget.remaining()} | "
              f"Budget: {self.budget.usage_percentage():.1f}% used")
        
        return response
    
    def _estimate_tokens(self, kwargs: Dict) -> int:
        """Estimate tokens for a request."""
        # Use the count endpoint for accurate estimation
        count_result = self.client.responses.input_tokens.count(**kwargs)
        
        # Add estimated output tokens (or use max_output_tokens if specified)
        estimated_output = kwargs.get('max_output_tokens', 500)
        
        return count_result.input_tokens + estimated_output

# Example usage
budget = TokenBudget(max_tokens=5000)
budgeted_client = BudgetedClient(client, budget)

# Make multiple requests within budget
for i in range(3):
    try:
        response = budgeted_client.create_response(
            model="gpt-5-mini",
            input=f"What is interesting fact #{i+1} about space?",
            max_output_tokens=100
        )
        print(f"Response {i+1}: {response.output_text[:80]}...\n")
    except ValueError as e:
        print(f"❌ Budget limit reached: {e}")
        break

## 4. Multi-Turn Conversation Builder

Most agents need to maintain context across multiple turns. This pattern provides a clean interface for building conversations, managing history, and handling context windows - the foundation of any chat agent.

In [ ]:
class ConversationBuilder:
    """Build and manage multi-turn conversations."""
    
    def __init__(
        self,
        client: OpenAI,
        model: str,
        instructions: Optional[str] = None,
        max_history: int = 10
    ):
        self.client = client
        self.model = model
        self.instructions = instructions
        self.max_history = max_history
        self.history: List[Dict[str, str]] = []
        self.last_response_id: Optional[str] = None
    
    def send(self, user_input: str, **kwargs) -> str:
        """Send a message and get response."""
        # Create response with history
        response = self.client.responses.create(
            model=self.model,
            input=user_input,
            instructions=self.instructions if not self.last_response_id else None,
            previous_response_id=self.last_response_id,
            **kwargs
        )
        
        # Update history
        self.history.append({
            "role": "user",
            "content": user_input,
            "timestamp": datetime.now().isoformat()
        })
        self.history.append({
            "role": "assistant",
            "content": response.output_text,
            "timestamp": datetime.now().isoformat(),
            "tokens": response.usage.total_tokens
        })
        
        # Trim history if needed
        if len(self.history) > self.max_history * 2:  # * 2 for user + assistant
            self.history = self.history[-(self.max_history * 2):]
        
        self.last_response_id = response.id
        
        return response.output_text
    
    def get_history(self) -> List[Dict[str, str]]:
        """Get conversation history."""
        return self.history
    
    def reset(self) -> None:
        """Reset conversation."""
        self.history = []
        self.last_response_id = None
    
    def format_history(self) -> str:
        """Format history as readable text."""
        formatted = []
        for msg in self.history:
            role = "User" if msg["role"] == "user" else "Assistant"
            formatted.append(f"{role}: {msg['content']}")
        return "\n\n".join(formatted)

# Example usage: Building a coding tutor agent
tutor = ConversationBuilder(
    client,
    model="gpt-5-mini",
    instructions="You are a patient coding tutor. Explain concepts clearly and provide examples.",
    max_history=5
)

# Simulate a conversation
questions = [
    "What is a variable in Python?",
    "Can you show me an example?",
    "What's the difference between a list and a tuple?"
]

for question in questions:
    print(f"\n🧑 User: {question}")
    response = tutor.send(question)
    print(f"🤖 Tutor: {response[:150]}...")

# Show conversation summary
print(f"\n📊 Conversation stats:")
print(f"   Total turns: {len(tutor.history) // 2}")
print(f"   Total tokens: {sum(msg.get('tokens', 0) for msg in tutor.history)}")

## 5. Error Handling Wrapper

Robust error handling is essential for production agents. This pattern provides a comprehensive wrapper that handles different error types, provides meaningful feedback, and ensures graceful degradation.

In [ ]:
from enum import Enum
from typing import Union

class ErrorSeverity(Enum):
    """Error severity levels."""
    LOW = "low"
    MEDIUM = "medium"
    HIGH = "high"
    CRITICAL = "critical"

@dataclass
class ErrorResult:
    """Structured error result."""
    error_type: str
    message: str
    severity: ErrorSeverity
    retryable: bool
    suggestion: Optional[str] = None

@dataclass
class SuccessResult:
    """Successful response result."""
    response: Any
    text: str
    tokens: int

Result = Union[SuccessResult, ErrorResult]

def safe_create_response(
    client: OpenAI,
    **kwargs
) -> Result:
    """
    Create response with comprehensive error handling.
    
    Returns:
        Either SuccessResult or ErrorResult
    """
    try:
        response = client.responses.create(**kwargs)
        
        # Check for failed status
        if response.status == "failed":
            return ErrorResult(
                error_type="ResponseFailed",
                message=f"Response failed: {response.error}",
                severity=ErrorSeverity.HIGH,
                retryable=True,
                suggestion="Check input parameters and try again"
            )
        
        return SuccessResult(
            response=response,
            text=response.output_text,
            tokens=response.usage.total_tokens
        )
    
    except RateLimitError as e:
        return ErrorResult(
            error_type="RateLimit",
            message="Rate limit exceeded",
            severity=ErrorSeverity.MEDIUM,
            retryable=True,
            suggestion="Wait a moment and retry with exponential backoff"
        )
    
    except APITimeoutError as e:
        return ErrorResult(
            error_type="Timeout",
            message="Request timed out",
            severity=ErrorSeverity.MEDIUM,
            retryable=True,
            suggestion="Retry the request or reduce complexity"
        )
    
    except APIError as e:
        return ErrorResult(
            error_type="APIError",
            message=str(e),
            severity=ErrorSeverity.HIGH,
            retryable=True,
            suggestion="Check API status and retry"
        )
    
    except ValueError as e:
        return ErrorResult(
            error_type="ValueError",
            message=str(e),
            severity=ErrorSeverity.HIGH,
            retryable=False,
            suggestion="Check input parameters"
        )
    
    except Exception as e:
        return ErrorResult(
            error_type="Unknown",
            message=str(e),
            severity=ErrorSeverity.CRITICAL,
            retryable=False,
            suggestion="Review error details and contact support if needed"
        )

# Example usage
result = safe_create_response(
    client,
    model="gpt-5-mini",
    input="Explain quantum computing briefly."
)

if isinstance(result, SuccessResult):
    print(f"✅ Success!")
    print(f"Response: {result.text[:100]}...")
    print(f"Tokens: {result.tokens}")
else:
    print(f"❌ Error: {result.error_type}")
    print(f"Message: {result.message}")
    print(f"Severity: {result.severity.value}")
    print(f"Retryable: {result.retryable}")
    if result.suggestion:
        print(f"💡 Suggestion: {result.suggestion}")

## 6. Function Calling Pattern

Function calling enables agents to interact with external systems and tools. This pattern shows how to define tools, handle function calls, and execute them - essential for building agents that can take actions.

**Note:** The calculator example uses a simplified approach for demonstration. In production, use a safe expression evaluator library like `simpleeval` or `asteval` instead of Python's `eval()`.

In [ ]:
class ToolRegistry:
    """Registry for managing available tools/functions."""
    
    def __init__(self):
        self.tools: Dict[str, Callable] = {}
        self.tool_definitions: List[Dict] = []
    
    def register(self, name: str, description: str, parameters: Dict):
        """Decorator to register a tool."""
        def decorator(func: Callable):
            self.tools[name] = func
            self.tool_definitions.append({
                "type": "function",
                "function": {
                    "name": name,
                    "description": description,
                    "parameters": parameters
                }
            })
            return func
        return decorator
    
    def execute(self, name: str, arguments: Dict) -> Any:
        """Execute a registered tool."""
        if name not in self.tools:
            raise ValueError(f"Tool '{name}' not found")
        return self.tools[name](**arguments)
    
    def get_definitions(self) -> List[Dict]:
        """Get tool definitions for API."""
        return self.tool_definitions

# Create registry and register tools
registry = ToolRegistry()

@registry.register(
    name="get_weather",
    description="Get current weather for a location",
    parameters={
        "type": "object",
        "properties": {
            "location": {
                "type": "string",
                "description": "City name or location"
            },
            "units": {
                "type": "string",
                "enum": ["celsius", "fahrenheit"],
                "description": "Temperature units"
            }
        },
        "required": ["location"]
    }
)
def get_weather(location: str, units: str = "celsius") -> str:
    """Simulate weather API call."""
    # In production, this would call a real weather API
    return json.dumps({
        "location": location,
        "temperature": 22,
        "units": units,
        "condition": "Partly cloudy"
    })

@registry.register(
    name="calculate",
    description="Perform simple mathematical calculations (addition, subtraction, multiplication, division)",
    parameters={
        "type": "object",
        "properties": {
            "operation": {
                "type": "string",
                "enum": ["add", "subtract", "multiply", "divide"],
                "description": "Mathematical operation to perform"
            },
            "a": {
                "type": "number",
                "description": "First number"
            },
            "b": {
                "type": "number",
                "description": "Second number"
            }
        },
        "required": ["operation", "a", "b"]
    }
)
def calculate(operation: str, a: float, b: float) -> str:
    """Safely perform basic mathematical operations."""
    try:
        operations = {
            "add": lambda x, y: x + y,
            "subtract": lambda x, y: x - y,
            "multiply": lambda x, y: x * y,
            "divide": lambda x, y: x / y if y != 0 else "Error: Division by zero"
        }
        
        result = operations[operation](a, b)
        return json.dumps({"result": result, "operation": operation, "a": a, "b": b})
    except Exception as e:
        return json.dumps({"error": str(e)})

def run_agent_with_tools(
    client: OpenAI,
    registry: ToolRegistry,
    user_input: str,
    model: str = "gpt-5-mini",
    max_iterations: int = 5
) -> str:
    """
    Run agent with tool calling capability.
    
    Handles the full tool calling loop:
    1. Send user input
    2. Check for tool calls
    3. Execute tools
    4. Send results back
    5. Get final response
    """
    previous_response_id = None
    
    for iteration in range(max_iterations):
        print(f"\n🔄 Iteration {iteration + 1}")
        
        # Create response with tools
        response = client.responses.create(
            model=model,
            input=user_input if iteration == 0 else None,
            tools=registry.get_definitions(),
            previous_response_id=previous_response_id
        )
        
        previous_response_id = response.id
        
        # Check for tool calls in output
        tool_calls = []
        for output_item in response.output:
            if output_item.type == "function_call":
                tool_calls.append(output_item)
        
        if not tool_calls:
            # No tool calls, we have the final response
            print("✅ Final response received")
            return response.output_text
        
        # Execute tool calls
        print(f"🔧 Executing {len(tool_calls)} tool(s)...")
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            arguments = json.loads(tool_call.function.arguments)
            
            print(f"   Calling: {function_name}({arguments})")
            result = registry.execute(function_name, arguments)
            print(f"   Result: {result[:80]}...")
            
            # Send tool result back to the model
            # Note: In the actual Responses API, you would send the tool results
            # as input in the next iteration
    
    return "Max iterations reached"

# Example usage
result = run_agent_with_tools(
    client,
    registry,
    user_input="What's the weather in San Francisco? Also calculate 15 multiplied by 24."
)

print(f"\n📝 Final Response:\n{result}")

## 7. Structured Output Parser

When building agents that need to return data in specific formats (JSON, lists, etc.), structured outputs ensure reliability. This pattern shows how to enforce and validate structured responses - crucial for agents that integrate with other systems.

In [ ]:
from typing import Type, TypeVar
from pydantic import BaseModel, ValidationError

T = TypeVar('T', bound=BaseModel)

def create_structured_response(
    client: OpenAI,
    model: str,
    input_text: str,
    schema: Type[T],
    instructions: Optional[str] = None,
    **kwargs
) -> T:
    """
    Create response with structured output and validation.
    
    Args:
        client: OpenAI client
        model: Model to use
        input_text: User input
        schema: Pydantic model class for validation
        instructions: Optional instructions
        **kwargs: Additional parameters
    
    Returns:
        Validated instance of schema
    """
    # Convert Pydantic model to JSON schema
    json_schema = schema.model_json_schema()
    
    response = client.responses.create(
        model=model,
        input=input_text,
        instructions=instructions,
        text={
            "type": "json_schema",
            "json_schema": {
                "name": schema.__name__,
                "schema": json_schema,
                "strict": True
            }
        },
        **kwargs
    )
    
    # Parse and validate response
    try:
        # Extract JSON from output
        json_text = response.output_text
        data = json.loads(json_text)
        
        # Validate with Pydantic
        validated = schema(**data)
        return validated
    
    except (json.JSONDecodeError, ValidationError) as e:
        raise ValueError(f"Failed to parse structured output: {e}")

# Define schemas for different use cases
class SentimentAnalysis(BaseModel):
    """Structured output for sentiment analysis."""
    text: str
    sentiment: str  # positive, negative, neutral
    confidence: float
    key_phrases: List[str]

class TaskExtraction(BaseModel):
    """Extract tasks from text."""
    task: str
    priority: str  # high, medium, low
    due_date: Optional[str] = None
    assignee: Optional[str] = None

class TaskList(BaseModel):
    """List of extracted tasks."""
    tasks: List[TaskExtraction]
    total_count: int

# Example 1: Sentiment Analysis
print("Example 1: Sentiment Analysis")
sentiment = create_structured_response(
    client,
    model="gpt-5-mini",
    input_text="This product is absolutely amazing! Best purchase I've made all year.",
    schema=SentimentAnalysis,
    instructions="Analyze the sentiment of the given text."
)

print(f"Sentiment: {sentiment.sentiment}")
print(f"Confidence: {sentiment.confidence}")
print(f"Key phrases: {', '.join(sentiment.key_phrases)}")

# Example 2: Task Extraction
print("\nExample 2: Task Extraction")
email_text = """
Hi team,
We need to complete the user auth by Friday. 
Also, someone should review the API docs (low priority).
John, can you deploy the staging environment ASAP?
"""

tasks = create_structured_response(
    client,
    model="gpt-5-mini",
    input_text=email_text,
    schema=TaskList,
    instructions="Extract all tasks from the email with priorities and assignees."
)

print(f"Found {tasks.total_count} tasks:")
for task in tasks.tasks:
    print(f"  [{task.priority}] {task.task}")
    if task.assignee:
        print(f"    Assignee: {task.assignee}")
    if task.due_date:
        print(f"    Due: {task.due_date}")

## 8. Cost Tracking Utility

Understanding costs is essential for production agents. This pattern tracks token usage, estimates costs based on model pricing, and provides reports - critical for managing budgets in production systems.

In [ ]:
@dataclass
class ModelPricing:
    """Pricing per million tokens."""
    input_price: float  # per million input tokens
    output_price: float  # per million output tokens
    cached_input_price: float = 0.0  # per million cached tokens

# Current pricing (update as needed)
PRICING = {
    "gpt-5-mini": ModelPricing(input_price=0.10, output_price=0.40),
    "gpt-5": ModelPricing(input_price=1.00, output_price=4.00),
}

class CostTracker:
    """Track costs across multiple API calls."""
    
    def __init__(self):
        self.calls: List[Dict] = []
        self.total_input_tokens = 0
        self.total_output_tokens = 0
        self.total_cost = 0.0
    
    def track_response(
        self,
        response,
        model: str,
        metadata: Optional[Dict] = None
    ) -> Dict:
        """Track a response and calculate cost."""
        usage = response.usage
        pricing = PRICING.get(model)
        
        if not pricing:
            raise ValueError(f"No pricing data for model: {model}")
        
        # Calculate costs
        input_cost = (usage.input_tokens / 1_000_000) * pricing.input_price
        output_cost = (usage.output_tokens / 1_000_000) * pricing.output_price
        total_cost = input_cost + output_cost
        
        # Handle cached tokens if available
        cached_cost = 0.0
        if hasattr(usage, 'input_tokens_details'):
            cached = usage.input_tokens_details.cached_tokens
            if cached > 0:
                cached_cost = (cached / 1_000_000) * pricing.cached_input_price
        
        call_data = {
            "timestamp": datetime.now().isoformat(),
            "model": model,
            "response_id": response.id,
            "input_tokens": usage.input_tokens,
            "output_tokens": usage.output_tokens,
            "total_tokens": usage.total_tokens,
            "input_cost": input_cost,
            "output_cost": output_cost,
            "cached_cost": cached_cost,
            "total_cost": total_cost,
            "metadata": metadata or {}
        }
        
        self.calls.append(call_data)
        self.total_input_tokens += usage.input_tokens
        self.total_output_tokens += usage.output_tokens
        self.total_cost += total_cost
        
        return call_data
    
    def get_summary(self) -> Dict:
        """Get cost summary."""
        return {
            "total_calls": len(self.calls),
            "total_input_tokens": self.total_input_tokens,
            "total_output_tokens": self.total_output_tokens,
            "total_tokens": self.total_input_tokens + self.total_output_tokens,
            "total_cost": self.total_cost,
            "avg_cost_per_call": self.total_cost / len(self.calls) if self.calls else 0,
        }
    
    def print_report(self):
        """Print detailed cost report."""
        summary = self.get_summary()
        
        print("\n" + "="*60)
        print("COST REPORT")
        print("="*60)
        print(f"Total API Calls: {summary['total_calls']}")
        print(f"Total Tokens: {summary['total_tokens']:,}")
        print(f"  Input: {summary['total_input_tokens']:,}")
        print(f"  Output: {summary['total_output_tokens']:,}")
        print(f"\nTotal Cost: ${summary['total_cost']:.4f}")
        print(f"Average Cost/Call: ${summary['avg_cost_per_call']:.4f}")
        print("\nRecent Calls:")
        
        for call in self.calls[-5:]:
            print(f"  {call['timestamp'][:19]} | "
                  f"{call['model']:12} | "
                  f"{call['total_tokens']:5} tokens | "
                  f"${call['total_cost']:.4f}")
        
        print("="*60 + "\n")

# Example usage
tracker = CostTracker()

# Make several API calls
test_prompts = [
    "What is machine learning?",
    "Explain neural networks briefly.",
    "What are transformers in AI?"
]

for i, prompt in enumerate(test_prompts):
    response = client.responses.create(
        model="gpt-5-mini",
        input=prompt,
        max_output_tokens=100
    )
    
    call_data = tracker.track_response(
        response,
        model="gpt-5-mini",
        metadata={"prompt_index": i}
    )
    
    print(f"Call {i+1}: {call_data['total_tokens']} tokens, ${call_data['total_cost']:.4f}")

# Print final report
tracker.print_report()

## 9. Context Window Manager

Long conversations can exceed context windows. This pattern manages conversation history intelligently, using strategies like truncation, summarization, and sliding windows - essential for agents handling extended interactions.

In [ ]:
class TruncationStrategy(Enum):
    """Strategies for handling context overflow."""
    SLIDING_WINDOW = "sliding_window"  # Keep most recent messages
    SUMMARIZE = "summarize"  # Summarize old messages
    HYBRID = "hybrid"  # Combine strategies

class ContextWindowManager:
    """Manage conversation context to stay within token limits."""
    
    def __init__(
        self,
        client: OpenAI,
        max_tokens: int = 4000,
        strategy: TruncationStrategy = TruncationStrategy.SLIDING_WINDOW,
        buffer_tokens: int = 500
    ):
        self.client = client
        self.max_tokens = max_tokens
        self.strategy = strategy
        self.buffer_tokens = buffer_tokens
        self.messages: List[Dict] = []
        self.summary: Optional[str] = None
    
    def add_message(self, role: str, content: str):
        """Add a message to context."""
        self.messages.append({"role": role, "content": content})
    
    def estimate_tokens(self, messages: List[Dict]) -> int:
        """Estimate token count for messages."""
        # Rough estimation: ~4 chars per token
        total_chars = sum(len(msg['content']) for msg in messages)
        return total_chars // 4
    
    def apply_sliding_window(self, messages: List[Dict]) -> List[Dict]:
        """Keep most recent messages that fit in context."""
        result = []
        current_tokens = 0
        
        # Process messages in reverse (most recent first)
        for msg in reversed(messages):
            msg_tokens = self.estimate_tokens([msg])
            if current_tokens + msg_tokens <= self.max_tokens - self.buffer_tokens:
                result.insert(0, msg)
                current_tokens += msg_tokens
            else:
                break
        
        return result
    
    def summarize_messages(self, messages: List[Dict]) -> str:
        """Summarize old messages to save tokens."""
        # Combine messages into text
        text = "\n".join([f"{msg['role']}: {msg['content']}" for msg in messages])
        
        # Create summary using the API
        response = self.client.responses.create(
            model="gpt-5-mini",
            input=f"Summarize this conversation concisely:\n\n{text}",
            max_output_tokens=200
        )
        
        return response.output_text
    
    def get_managed_context(self) -> List[Dict]:
        """Get context managed according to strategy."""
        current_tokens = self.estimate_tokens(self.messages)
        
        # If within limits, return as-is
        if current_tokens <= self.max_tokens - self.buffer_tokens:
            return self.messages
        
        print(f"⚠️  Context overflow: {current_tokens} tokens. Applying {self.strategy.value}...")
        
        if self.strategy == TruncationStrategy.SLIDING_WINDOW:
            return self.apply_sliding_window(self.messages)
        
        elif self.strategy == TruncationStrategy.SUMMARIZE:
            # Keep recent messages, summarize old ones
            recent_count = 4  # Keep last 4 messages
            old_messages = self.messages[:-recent_count]
            recent_messages = self.messages[-recent_count:]
            
            if old_messages:
                summary = self.summarize_messages(old_messages)
                return [
                    {"role": "system", "content": f"Previous conversation summary: {summary}"},
                    *recent_messages
                ]
            return recent_messages
        
        elif self.strategy == TruncationStrategy.HYBRID:
            # Summarize very old, keep middle, retain recent
            if len(self.messages) > 10:
                very_old = self.messages[:len(self.messages)//2]
                recent = self.messages[len(self.messages)//2:]
                
                summary = self.summarize_messages(very_old)
                managed = [
                    {"role": "system", "content": f"Summary: {summary}"},
                    *self.apply_sliding_window(recent)
                ]
                return managed
            
            return self.apply_sliding_window(self.messages)
        
        return self.messages

# Example usage
manager = ContextWindowManager(
    client,
    max_tokens=2000,
    strategy=TruncationStrategy.SLIDING_WINDOW
)

# Simulate a long conversation
conversation_turns = [
    ("user", "What is Python?"),
    ("assistant", "Python is a high-level programming language..."),
    ("user", "What are its main features?"),
    ("assistant", "Python's main features include..."),
    ("user", "How do I install it?"),
    ("assistant", "You can install Python by..."),
    ("user", "What are virtual environments?"),
    ("assistant", "Virtual environments are isolated Python installations..."),
]

for role, content in conversation_turns:
    # Add some filler to simulate longer messages
    filled_content = content + (" Additional context." * 50)
    manager.add_message(role, filled_content)

# Get managed context
managed = manager.get_managed_context()
print(f"\nOriginal messages: {len(manager.messages)}")
print(f"Managed messages: {len(managed)}")
print(f"Estimated tokens: {manager.estimate_tokens(managed)}")

## 10. Response Caching Pattern

Caching responses can significantly reduce costs and latency for repeated queries. This pattern implements a simple but effective caching layer - useful for agents that handle similar queries or have frequently accessed information.

In [ ]:
import hashlib
from datetime import timedelta

@dataclass
class CachedResponse:
    """Cached response with metadata."""
    text: str
    response_id: str
    timestamp: datetime
    tokens: int
    model: str

class ResponseCache:
    """Cache responses to reduce API calls and costs."""
    
    def __init__(self, ttl: timedelta = timedelta(hours=24)):
        self.cache: Dict[str, CachedResponse] = {}
        self.ttl = ttl
        self.hits = 0
        self.misses = 0
        self.tokens_saved = 0
    
    def _generate_key(self, model: str, input_text: str, **kwargs) -> str:
        """Generate cache key from parameters."""
        # Include important parameters that affect output
        key_params = {
            "model": model,
            "input": input_text,
            "temperature": kwargs.get("temperature", 1.0),
            "max_tokens": kwargs.get("max_output_tokens"),
            "instructions": kwargs.get("instructions", "")
        }
        
        # Create hash
        key_str = json.dumps(key_params, sort_keys=True)
        return hashlib.sha256(key_str.encode()).hexdigest()[:16]
    
    def _is_expired(self, cached: CachedResponse) -> bool:
        """Check if cached response is expired."""
        age = datetime.now() - cached.timestamp
        return age > self.ttl
    
    def get(self, model: str, input_text: str, **kwargs) -> Optional[CachedResponse]:
        """Get cached response if available."""
        key = self._generate_key(model, input_text, **kwargs)
        
        if key in self.cache:
            cached = self.cache[key]
            
            if not self._is_expired(cached):
                self.hits += 1
                self.tokens_saved += cached.tokens
                return cached
            else:
                # Remove expired entry
                del self.cache[key]
        
        self.misses += 1
        return None
    
    def set(self, response, model: str, input_text: str, **kwargs):
        """Cache a response."""
        key = self._generate_key(model, input_text, **kwargs)
        
        self.cache[key] = CachedResponse(
            text=response.output_text,
            response_id=response.id,
            timestamp=datetime.now(),
            tokens=response.usage.total_tokens,
            model=model
        )
    
    def get_stats(self) -> Dict:
        """Get cache statistics."""
        total_requests = self.hits + self.misses
        hit_rate = (self.hits / total_requests * 100) if total_requests > 0 else 0
        
        return {
            "hits": self.hits,
            "misses": self.misses,
            "hit_rate": hit_rate,
            "tokens_saved": self.tokens_saved,
            "cached_entries": len(self.cache)
        }
    
    def clear(self):
        """Clear all cached responses."""
        self.cache.clear()

class CachedClient:
    """OpenAI client wrapper with caching."""
    
    def __init__(self, client: OpenAI, cache: ResponseCache):
        self.client = client
        self.cache = cache
    
    def create_response(self, model: str, input: str, **kwargs):
        """Create response with caching."""
        # Check cache first
        cached = self.cache.get(model, input, **kwargs)
        
        if cached:
            print(f"✓ Cache hit! Saved {cached.tokens} tokens")
            return cached
        
        # Cache miss - call API
        print(f"⊗ Cache miss - calling API")
        response = self.client.responses.create(
            model=model,
            input=input,
            **kwargs
        )
        
        # Cache the response
        self.cache.set(response, model, input, **kwargs)
        
        return response

# Example usage
cache = ResponseCache(ttl=timedelta(minutes=30))
cached_client = CachedClient(client, cache)

# Make the same query multiple times
query = "What is the capital of France?"

print("Request 1:")
response1 = cached_client.create_response(
    model="gpt-5-mini",
    input=query
)
print(f"Response: {response1.text if isinstance(response1, CachedResponse) else response1.output_text}\n")

print("Request 2 (same query):")
response2 = cached_client.create_response(
    model="gpt-5-mini",
    input=query
)
print(f"Response: {response2.text if isinstance(response2, CachedResponse) else response2.output_text}\n")

print("Request 3 (same query):")
response3 = cached_client.create_response(
    model="gpt-5-mini",
    input=query
)

# Show cache statistics
stats = cache.get_stats()
print(f"\n📊 Cache Statistics:")
print(f"   Hit rate: {stats['hit_rate']:.1f}%")
print(f"   Hits: {stats['hits']}, Misses: {stats['misses']}")
print(f"   Tokens saved: {stats['tokens_saved']}")
print(f"   Cached entries: {stats['cached_entries']}")

## Summary

This notebook presented 10 essential patterns for building production-ready agentic applications:

1. **Streaming Response Handler** - Real-time feedback with callbacks
2. **Retry Logic** - Reliable API calls with exponential backoff
3. **Token Budget Manager** - Cost control and usage tracking
4. **Multi-Turn Conversation Builder** - Managing conversation state
5. **Error Handling Wrapper** - Robust error management
6. **Function Calling Pattern** - Tool integration for agents
7. **Structured Output Parser** - Validated, structured responses
8. **Cost Tracking Utility** - Budget monitoring and reporting
9. **Context Window Manager** - Smart history management
10. **Response Caching** - Performance and cost optimization

Each pattern addresses real production challenges and can be adapted for your specific use case. Combine these patterns to build sophisticated agents that are reliable, cost-effective, and maintainable.